3D энкодер- декодер. Отличие от предыдущей версии - вместо растра точек на вход сети подать вектора лендмарков в виде набора координат и длин.
к MDWL добавлены еще 2 лендмарка BCP, FAP - размер энкодера вырос на 3 +3 значения итого 12 значений* 16 зубов = 192.

In [1]:
# from keras.datasets import mnist
import numpy as np
# from jaw_gen import *
import tensorflow 
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import tensorflow.keras.backend as K
from tensorflow.keras.layers import Lambda
import cv2

from tqdm import tqdm
# основное отличие - Landmark_gen больше не генерит картинки. только вектора. 
# и у него теперь есть 3D рисовалка, которая есть независимый метод
# генератор теперь в другом файле, генерит дополнительно 2 лендмарка
from jaw_gen3d_3lm import Landmark_gen 
from csv_parser import set_gen_fr_csv

# готовим СИНТЕТИЧЕСКИЕ входные данные 

In [2]:

SYNT = 0                     # True - учим на синтетике, False - на реальных

# готовим СИНТЕТИЧЕСКИЕ входные данные 
inst_ =                 Landmark_gen()
data_len =              5000
dataset =               []
dataset_spoiled =       []
dataset_vec =           []
dataset_vec_spoiled =   []

if SYNT:
    for i in range(data_len):
        scale =  9 + np.random.sample()*3     # это для размера 200, диапазон от 14 до 19 для размера 400
        factor = 2.2 + np.random.sample()/4   # это для размера 200, диапазон 2.5 - 2.7 для размера 400 
        # name = f"scale {scale:.4}  factor {factor:.3}"
        # сдвиги и генерация остальных лендмарков по умолчанию включены. 
        vec, vec_spoiled = inst_.image_gen(scale=scale, factor=factor, spoiledMDW=True, shiftMDW=True)

        # собираем датасет из векторов 
        dataset_vec.append(vec)
        dataset_vec_spoiled.append(vec_spoiled)
    # print (f"dataset_vec {dataset_vec[:1]}")
    print (f"dataset_vec len {len(dataset_vec)} ")
    print (f"dataset_vec_spoiled len {len(dataset_vec_spoiled)} ")

    # преобразуем датасет в numpy массив
    dataset_vec =           np.array(dataset_vec,           dtype="float32") # /2550 тогда не учится ничего 
    dataset_vec_spoiled =   np.array(dataset_vec_spoiled,   dtype="float32") # /2550

# print (f"datraset vec {dataset_vec[:1]}")


# готовим данные из файла CSV

In [3]:
# готовим датасет из файла csv
# csv_fpath = 'C:/Users/Anton/Projects/jaw_encoder/csv/input_004.csv'
csv_fpath = 'C:/Users/Anton/Projects/jaw_encoder/csv/test10.csv'

# выбираем данные из файла
if not SYNT:
    dataset_vec_spoiled, dataset_vec = set_gen_fr_csv(csv_fpath) # возвращает np массивы сначала T0


# dataset_vec = dataset_vec_spoiled # дичь, дичь!!!
# dataset_vec_spoiled *= 200
# dataset_vec         *= 200
data_len = len(dataset_vec)
print (f"data lenght {data_len}")
[print (f"{i}val {dataset_vec[i][5]} \n {i}spoil {dataset_vec_spoiled[i][5]}") for i in range(len(dataset_vec))]
# print (f"val {dataset_vec[3][5]} \n {dataset_vec_spoiled[3][5]}") 
print (f"!")

data lenght 30
0val [ -9.22192    2.95499    0.847423 -13.2823     8.40958    2.01664
 -11.6258     5.29081    0.64333  -12.5505     4.72119   -4.61977 ] 
 0spoil [ -7.96088    3.88473    1.06298  -12.1189     9.29794    2.07035
 -10.4698     6.10762    0.862325 -11.865      4.96074   -4.1991  ]
1val [-10.5864     2.86047    0.125182 -15.0495     6.21431    0.3302
 -12.7699     4.42191    0.146344 -13.573      3.47264   -4.19148 ] 
 1spoil [-10.5864     2.86047    0.125182 -15.0495     6.21431    0.3302
 -12.7699     4.42191    0.146344 -13.573      3.47264   -4.19148 ]
2val [-11.4231    1.30329   1.72989 -16.155     6.0084    1.76171 -13.4668
   3.75399   1.78117 -14.8039    3.00259  -3.39116] 
 2spoil [-11.5327    1.48368   1.4604  -16.2646    6.18879   1.49222 -13.5764
   3.93438   1.51168 -14.9136    3.18299  -3.66065]
3val [-13.6306     5.01497    0.157642 -17.4609     9.37813    0.601678
 -15.3423     7.0433     0.239744 -16.7157     6.6556    -4.1641  ] 
 3spoil [-13.5344     5.

In [4]:
# разобьем датасет на тренировочную и тестовую части
set_divider_ = round(data_len*0.8)

# векторный датасет
x_train_vec =   dataset_vec[:set_divider_]
x_test_vec =    dataset_vec[set_divider_:]
# print (f"length vec {len(x_train_vec)} - {len(x_test_vec)}")

x_train_vec_spoiled =   dataset_vec_spoiled[:set_divider_]
x_test_vec_spoiled =    dataset_vec_spoiled[set_divider_:]
# print (f"vec [0]{x_test_vec[0]}")
print (f"length vec         {len(x_train_vec)} - {len(x_test_vec)}")
print (f"length vec spoil   {len(x_train_vec_spoiled)} - {len(x_test_vec_spoiled)}")

length vec         24 - 6
length vec spoil   24 - 6


In [5]:
# Размерности слоев энкодера 
dense_dim = 192             # размерность полносвязного слоя энкодера 16 зубов * 12 значений.
dim_code = dense_dim//6     # размерность кодированного слоя

In [6]:
def create_vec_ae3D():
    # 3d кодер - декодер содержит на входе 12*16=192 значений.
    activation = 'elu' 
    # activation = 'linear'
    inputs = Input(dense_dim,)
    WITH_BN = False # go with batcNormalization

    # encoder
    x = Dense(dense_dim, activation=activation)(inputs)
    x= BatchNormalization()(x)      if WITH_BN else x
    x = Dense(dense_dim, activation=activation)(x)
    x= BatchNormalization()(x)      if WITH_BN else x
    code = Dense(dim_code, activation='linear')(x)

    # decoder
    input_encoded = Input((dim_code,))
    x = Dense(dim_code, activation=activation)(input_encoded)
    x= BatchNormalization()(x)      if WITH_BN else x
    x = Dense(dense_dim, activation=activation)(x)
    x= BatchNormalization()(x)      if WITH_BN else x
    out = Dense(dense_dim, activation='linear')(x)
    
    # Модели
    encoder = Model(inputs, code, name="encoder")
    decoder = Model(input_encoded, out, name="decoder")
    autoencoder = Model(inputs, decoder(encoder(inputs)), name="autoencoder")
    return encoder, decoder, autoencoder

encoder, decoder, autoencoder = create_vec_ae3D()
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
# autoencoder.compile(Adam(0.0001), loss='mean_squared_error') 

# c_autoencoder.summary()

In [7]:
batch_size = 16

def create_denoising_model(autoencoder):
    
    inputs  = Input(batch_shape=(batch_size, dense_dim))
    inputs_spoiled  = Input(batch_shape=(batch_size, dense_dim)) # вход для кореженных зубьев

    denoiser_model = Model([inputs, inputs_spoiled], autoencoder(inputs_spoiled), name="denoiser")
    # return noiser, denoiser_model
    return denoiser_model


# noiser, denoiser_model = create_denoising_model(c_autoencoder)
denoiser_model = create_denoising_model(autoencoder)
denoiser_model.compile(optimizer='adam', loss='mean_squared_error')
# denoiser_model.compile(Adam(1e-4), loss='mean_squared_error')
# optimizer=tf.keras.optimizer.Adam(learning_rate=1e-3)
# keras.optimizers.Adam(learning_rate=1e-3)

In [8]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

def plot_digits(*args):
    # for a in args:
        # print (f"type args {type(a)}")
    args = [x.squeeze() for x in args]
    n = min([x.shape[0] for x in args])
    
    plt.figure(figsize=(2*n, 2*len(args)))
    for j in range(n):
            for i in range(len(args)):
                ax = plt.subplot(len(args), n, i*n + j + 1)
                plt.imshow(args[i][j])
                plt.gray()
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)

    plt.show()

In [9]:
# вместо фита будем делать  model.train_on_batch читобы смотреть как учится и куда движемся 
# хотя у фита много больше возможностей с колбеками
def train_on_batch(vec, vec_spoil, batch_size=batch_size, epochs=100):
    # подровняем массив картинок и обучающих векторов так, чтоб np.reshape не ругался - на несовпадение размерности
    # сделаем его длину кратной размеру батча
    
    vec =   vec [:len(x_train_vec)//batch_size*batch_size]
    vec =   np.reshape(vec, (-1, batch_size, dense_dim))           # x_test  = np.reshape(x_test,  (len(x_test),  200, 200, 1))
    
    # и массив кореженных тоже
    vec_spoil = vec_spoil   [:len(x_train_vec_spoiled)  //batch_size*batch_size]
    vec_spoil = np.reshape(vec_spoil,   (-1, batch_size, dense_dim)) 

    losses = [10e100] # потери для сравнения эпох будем тут сохранять 

    for epoch in range(1, epochs+1):
        # print(f"Epoch {epoch}")
        for i, _ in enumerate(vec):
            batch_vec =         vec[i] 
            batch_vec_spoiled = vec_spoil[i]

            loss = denoiser_model.train_on_batch([batch_vec, batch_vec_spoiled], batch_vec)

        # эпоха отучилась, построим что вышло.
        n = 10
        vecs = batch_vec                    # это последний батч в каждой эпохе.
        # print (f"1! vecs shape{vecs.shape}") # shape - (256, 64)
        noised_vecs = batch_vec_spoiled     
        # а картинки не участвуют в учении, количество бачей у них при делении датасета на бачи
        # с векторами разное, поэтому найти соответствующую картинку проблематично. 
        # noised_imgs = noiser.predict(imgs, batch_size=batch_size)

        # задача в том, чтобы картинке сопоставить спойленую картинку ( это не сложно из сгенерированной 
        # выборки сделать), но потом предиктить для этой картинки вектора. и рисовать их на новой картинке 
        # средствами opencv 
        encoded_vecs = encoder.predict(noised_vecs[:n],  batch_size=n)
        decoded_vecs = decoder.predict(encoded_vecs[:n], batch_size=n)

        # формируем картинки из предсказанных векторов
        vecs =              np.reshape(vecs,            (-1, 16, dense_dim//16))
        noised_vecs =       np.reshape(noised_vecs,     (-1, 16, dense_dim//16))
        predicted_vecs =    np.reshape(decoded_vecs,    (-1, 16, dense_dim//16))
        
        # print (f"vecs shape{vecs.shape} vecs[0] {vecs[0]}")
        if (epoch%200 ==0) or (epoch in [1,2, 5, 10, 30, 80, 150]):
            print(f"Epoch {epoch} loss -{loss}")
        # рисуем картинки и сохраняем модели
        if (epoch % 50 ==0) and (epoch >= 500):
            # print(f'Epoch {epoch}')
            # plot_digits(batch_x[:n], batch_x_spoil[:n], imgs_fr_vec[:n], imgs_spoil_fr_vec[:n], imgs_fr_decoded_vec[:n])            
            if loss < min(losses):
                losses.append(loss)
                # save only the best models
                #denoiser_model.save(f'models3D/denoiser{epoch}ep_loss__{loss:.3}__.h5')
                decoder.save(f'models3D/decoder{epoch}ep_loss__{loss:.3}__.h5')
                encoder.save(f'models3D/encoder{epoch}ep_loss__{loss:.3}__.h5')
        
train_on_batch(x_train_vec, x_train_vec_spoiled, batch_size=batch_size, epochs=5000)

Epoch 1 loss -131.56072998046875
Epoch 2 loss -125.65422821044922
Epoch 5 loss -107.82199096679688
Epoch 10 loss -61.37724304199219
Epoch 30 loss -12.55184555053711
Epoch 80 loss -2.963963031768799
Epoch 150 loss -0.5176236629486084
Epoch 200 loss -0.3304217755794525
Epoch 400 loss -0.13620923459529877


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x000001F6B8988828>
Traceback (most recent call last):
  File "C:\Python37\lib\site-packages\tensorflow\python\data\ops\iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "C:\Python37\lib\site-packages\tensorflow\python\ops\gen_dataset_ops.py", line 1263, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Epoch 600 loss -0.06228101626038551


In [ ]:
# c_autoencoder.save(f'models/c_autoencoder.h5'